<a href="https://colab.research.google.com/github/jcole75/ARC_Solver/blob/main/nlpARC_BART.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Experiment W - BART-LARGE-CNN
Model: bart-large-cnn
Dataset description: (1548 items)
ARC Train and Evaluation Data
ARC Excluding 100 test items from Evaluation (test items are the first 100 items in Evaluation)
YK hand crafted (10 items)
JC hand crafted (5 items)
Rigid and Half Generated (~800 items) AndreasK

*Evaluation dataset*: Test dataset from ARC. 

*Answer format example*: 24 5 4 7288 7989 8777 7787 8777.

*Answer description*: String length, Number of grid rows, Number of grid columns, Output grid.

*Results file description*: This notebook tries to parse the generated answer in different ways: minimal parsing, parsing by the predicted length, parsing by the predicted grid size, parsing by the length of the correct answer.  It tallies the number of correct answers in all of those categories in addition to the accuracy of the length prediction and output grid dimensions prediction. 

*Purpose*: Compare to regular large model (without the cnn summarization training).  That model was trained on colab. It used the same dataset and did better than the same model trained without the Rigid and Half Generated data.

Some code derived from: https://towardsdatascience.com/fine-tuning-the-bart-large-model-for-text-summarization-3c69e4c04582


In [ ]:
# only run this cell if you are in collab
#%pip install transformers -Uqq
#%pip install datasets -Uqq
#%pip install bert-score -Uqq
#%pip install sacremoses
#%pip install ohmeow-blurr -Uqq

In [ ]:
# Setting up the device for GPU usage
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "1"

from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)

cuda


In [ ]:
epoch_start_up=0 # number of epochs to train for testing the code
epoch_start=0 # starting epoch number for training
epoch_current_count=0 # tracks the current epoch number
epoch_end=100 # ending epoch number for training
epoch_step=10 # step size for epochs

batch_size = 1
batch_size_start=1
batch_size_end=0
lr = 3.1e-5 # learning rate

grid_starting_row = 3 # where the grid starts in the generated answer
output_contains_grid_size = True # if the output contains the grid size

experiment_folder='./'
# use the url to get the dataset if you don't already have it
training_data_file = 'train_data_arc_yk_jc_rigid800.csv' # https://drive.google.com/file/d/17Lr2AjvUgDUpylxqPYHWqtjNJhYEw2Cv/view?usp=sharing
validation_data_file = '100_test_items_evaluation_data.csv' # https://drive.google.com/file/d/1LTD3onc4oE7ZyE138eTHRX_eQ1qNA3ws/view?usp=sharing


In [ ]:
#use if on colab and you want to save data to Google Drive
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
import datasets
import pandas as pd
from fastai.text.all import *
#from transformers import *
# Importing the T5 modules from huggingface/transformers
from transformers import BartTokenizerFast, BartForConditionalGeneration


from blurr.text.data.all import *
from blurr.text.modeling.all import *

import nltk
nltk.download('punkt', quiet=True)

True

In [ ]:
#set the maximum input length for the model
max_input_length = 1024
#raw_data = datasets.load_dataset('cnn_dailymail', '3.0.0', split='train[:1%]')
#raw_data = datasets.load_dataset('cnn_dailymail', '3.0.0', split='train[:1%]')
df = pd.read_csv(f"{experiment_folder}{training_data_file}")
#df2 = pd.read_csv("https://mindware.mobi/evaluation_data.csv")
df2 = pd.read_csv(f"{experiment_folder}{validation_data_file}")
# filter out the rows with a token_count>max_input_length
df = df[df['token_count']<=max_input_length]
df2 = df2[df2['token_count']<=max_input_length]


In [ ]:
df.head()

,id,prompt,correct_answer,token_count,token_count_correct
1,42a15761_0,train input1 22202220222 20202220202 22202220222 20202220202 22202220222 22202020202 22202220222 output1 22202220222 22202020202 22202220222 22202020202 22202220222 20202220202 22202220222. input2 222022202220222 222020202020222 222022202220222 222020202020202 222022202220222 202020202020222 222022202220222 202022202020222 222022202220222 output2 222022202220222 222022202020202 222022202220222 202022202020202 222022202220222 222020202020202 222022202220222 222020202220202 222022202220222. input3 222022202220222 222020202220202 222022202220222 222020202020202 222022202220222 202022202020202...,219 11 19 2220222022202220222 2220222020202020202 2220222022202220222 2220222022202220202 2220222022202220222 2220202020202020202 2220222022202220222 2020222020202020202 2220222022202220222 2220202022202020202 2220222022202220222.,403,96
2,4364c1c4_0,train input1 888888888888 886666888888 886666888888 886666666888 886666666888 883333333888 883333333888 883333333888 883333333888 883338833888 888888888888 888888888888 888888888888 888888888888 888888888888 888888888888 output1 888888888888 866668888888 866668888888 866666668888 866666668888 888333333388 888333333388 888333333388 888333333388 888333883388 888888888888 888888888888 888888888888 888888888888 888888888888 888888888888. input2 111111111111 111111111111 114111444111 114111444111 114444444111 114444444111 112222222111 112222112111 111111111111 111111111111 111111111111 11111111...,195 14 13 8888888888888 8188188888888 8111188888888 8111188888888 8882222888888 8888888888888 8888888888888 8888888888888 8888888888888 8888822288888 8888822288888 8888888444888 8888888484888 8888888888888.,501,102
3,456873bc_0,train input1 0000002220022200222 0000002000020000200 0000022200222002220 0000022000220002200 0000000000000000000 3333333333333333333 3333333333333333333 3333333333333333333 3333333333333333333 0000000000000000000 0222002220022200000 0200002000020000000 2220022200222000000 2200022000220000000 0000000000000000000 0222002220000000000 0200002000000000000 2220022200000000000 2200022000000000000 output1 0000008220028200228 0000002000020000200 0000022200222002220 0000022000220002200 0000000000000000000 0000002220000000000 0000008000000000000 0000022200000000000 0000022000000000000 000000000000000...,379 19 19 8222028220228202228 2000020000200002000 2002020020200202002 2022020220202202022 0000000000000000000 2222000000000000000 8000000000000000000 2002000000000000000 2022000000000000000 0000000000000000000 2222000000000002222 2000000000000002000 8002000000000002008 2022000000000002022 0000000000000000000 2222000000222202222 2000000000200002000 2002000000200202002 8022000000208202028.,478,105
4,45737921_0,train input1 0000000000 0884000000 0484000000 0884000000 0000003220 0000003320 0000003220 0000000000 0036300000 0036300000 0033300000 0000000000 0000000000 output1 0000000000 0448000000 0848000000 0448000000 0000002330 0000002230 0000002330 0000000000 0063600000 0063600000 0066600000 0000000000 0000000000. input2 0000000 0585000 0585000 0888000 0000000 0000000 0000000 output2 0000000 0858000 0858000 0555000 0000000 0000000 0000000. input3 000000000000 011100000000 081800032300 088800022200 000000023200 000000000000 000611000000 000166000000 000166004450 000000004550 000000005550 0000000000...,79 8 9 000000000 023300000 022300000 023300000 000001110 000006660 000006110 000000000.,244,34
5,45bbe264_0,train input1 0000000000000000 0000000000000000 0000800000000000 0000000000000000 0000000000000000 0000000000000000 0000000000000000 0000000000000000 0000000000000000 0000000000500000 0000000000000000 0000000000000000 0000000000000000 0000000000000000 0000000000000000 0000000000000000 output1 0000800000500000 0000800000500000 8888888888288888 0000800000500000 0000800000500000 0000800000500000 0000800000500000 0000800000500000 0000800000500000 5555255555555555 0000800000500000 0000800000500000 0000800000500000 0000800000500000

# Performs Inference, Data Parsing, and Item Stats

In [ ]:
# calculate z scores (absolute values) for a list of numbers
def z_score(numbers):
    mean = np.mean(numbers)
    std = np.std(numbers)
    return abs([(x - mean) / std for x in numbers])

# drop list elements after one that has a z score of length greater than the specified threshold
def drop_array_elements_after_z_score_threshold(string_list, threshold):
    # create a list of string lengths
    string_lengths = [len(x) for x in string_list]
    # calculate z scores for each element in the list
    z_scores = z_score(string_lengths)
    # drop list elements after one that has a z score of length greater than the specified threshold
    for i in range(len(z_scores)):
        if z_scores[i] > threshold:
            string_list = string_list[:i]
            break
    return string_list

#more likely to be correct with variable sized output for rows
def trim_rows_to_smallest(grid_rows):
    #trim the strings in grid_rows to the smallest length
    smallest_length = len(grid_rows[0])
    for i in range(len(grid_rows)):
        if len(grid_rows[i]) < smallest_length:
            smallest_length = len(grid_rows[i])
    for i in range(len(grid_rows)):
        grid_rows[i] = grid_rows[i][:smallest_length]
    return grid_rows

# trim / pad to grid size
def trim_pad_to_grid_size(grid_rows, grid_row_count, grid_column_count):
    #trim the strings in grid_rows to the smallest length
    if grid_row_count>30:
        grid_row_count=30
    if grid_column_count>30:
        grid_column_count=30
    # drop rows that are beyond the row count
    grid_rows = grid_rows[:grid_row_count]
    for i in range(grid_row_count):
        if i<len(grid_rows):
            grid_rows[i] = grid_rows[i][:grid_column_count]
            # pad out to grid_column_count with 0'SCRIPT_INFO
            zeros_to_add = ''.join(['0' for x in range(grid_column_count - len(grid_rows[i]))])
            grid_rows[i] = grid_rows[i] + zeros_to_add
        else:
            # add some empty row(s) if there are fewer than grid_row_count rows
            grid_rows.append(['0' for x in range(grid_column_count)])
            # join the rows into a single string
            grid_rows[i] = ''.join(grid_rows[i])
    return grid_rows

#remove empty arrays
def remove_empty_arrays(array):
    return [x for x in array if x != '']
    
#remove leading and trailing spaces from all strings in an array
def remove_leading_and_trailing_spaces(array):
    return [x.strip() for x in array]

#split the string by anything other than a space or a number. strip leading or trailing spaces. remove empty arrays.
def split_by_non_numbers_and_not_spaces(string):
    """return a list of numbers split by anything other than numbers and spaces"""
    return remove_leading_and_trailing_spaces(remove_empty_arrays(re.split(r'[^\d\s]', string)))

#parse the answers from the large model
def parse_answers_model1(outputs):
       #if output_val contains "." then get the first part of the string before the "."
    if "." in outputs[0]['summary_texts']:
        # make sure it is a string
        output_val = str(outputs[0]['summary_texts']).split(".")[0]
        #output_val=outputs[0]['summary_texts'].split(".")[0]
    else:
        output_val = str(outputs[0]['summary_texts'])
        #output_val=outputs[0]['summary_texts']
    # trim spaces
    output_val = output_val.strip()
    # check to see if it has the minimum number of words, which is grid_starting_row+2
    if len(output_val.split()) >= grid_starting_row+2:
        output_len = int(output_val.split(" ")[0]) #subtract 1 to account for the period that is removed
        # if the grid starting column is > 1 then get the words before the grid starting column
        if grid_starting_row > 1:
            other_preds = output_val.split(" ")[:grid_starting_row]
            # remove first element (the number)
            other_preds.pop(0)
        else:
            other_preds = []
        output_val = split_by_non_numbers_and_not_spaces(output_val)[0]
        output_val = output_val.split(" ")[grid_starting_row:]
        # reassemble the list of words
        output_val = " ".join(output_val)
        try:
            output_len = int(output_len)
        except ValueError:
            output_len = len(output_val)
        output_val_len_by_prediction=output_val[:output_len]
        output_rows_full_pred_len_limited = output_val_len_by_prediction.split(" ")
        output_rows_full_pred_len_limited=remove_empty_arrays(output_rows_full_pred_len_limited)
        output_rows = output_val.split(" ")
        #if the last element of output_rows is shorter than the first element, remove the last element.
        '''
        # if output_rows is a list
        if isinstance(output_rows, list):
            if len(output_rows[-1]) < len(output_rows[0]):
                output_rows = output_rows[:-1]
        '''
        if output_contains_grid_size and other_preds[0].isdigit() and other_preds[1].isdigit():
            # if the output contains the grid size, then trim the output to the grid size
            output_rows = trim_pad_to_grid_size(output_rows, int(other_preds[0]), int(other_preds[1]))
        else:
            output_rows = trim_rows_to_smallest(output_rows)
        output_rows_full_pred_len_limited=trim_rows_to_smallest(output_rows_full_pred_len_limited)

        #recombine the elements of output_rows into a single string.
        output_val_len_by_prediction = " ".join(output_rows_full_pred_len_limited)     
        output_val_by_grid_size_pred = " ".join(output_rows)
        #fix row lengths for the first answer
        output_rows_answer1 = output_val.split(" ")
        output_rows_answer1=trim_rows_to_smallest(output_rows_answer1)
        output_val = " ".join(output_rows_answer1) 
        error=False
    else:
        output_val_len_by_prediction=output_val
        output_len=len(output_val)
        # add 0's to other_preds for up to grid_starting_row-1
        other_preds = [0 for i in range(grid_starting_row)]
        output_val_by_grid_size_pred = output_val
        error=True
    return output_val, output_val_len_by_prediction, output_val_by_grid_size_pred, output_len, other_preds, error

def export_model_pickel(model, file_name):
    model.metrics = None
    model.export(fname=file_name)
    print("Model saved to "+file_name)

def import_model_pickel(file_name):
    return load_learner(fname=file_name)

def get_and_save_predictions(export_model_first = True):    
    if export_model_first: 
        learn.metrics = None
    picklefn=f"{experiment_folder}ft_cnndm_export_{batch_size_start}_bs_start_{batch_size_end}_{epoch_current_count}_epochs_per_bs_{lr}_lr_{len(df)}_items.pkl"
    if export_model_first: 
        export_model_pickel(learn, picklefn)
    inf_learn = import_model_pickel(picklefn)

    evaluation_files=[f"{experiment_folder}{validation_data_file}"]

    base_path=f"{experiment_folder}"
    sum_correct_answer=0
    sum_correct_answer_by_actual_len=0
    sum_correct_answer_len=0
    sum_correct_answer_pred_len=0
    sum_correct_answer_by_pred_grid_size=0
    sum_other_preds=[]
    for i in range(grid_starting_row-1):
        sum_other_preds.append(0)
    #iterate through the evaluation files
    for file in evaluation_files:
        data_to_evaluate=pd.read_csv(os.path.join(base_path, file))
        for i in range(len(df2)):
            arc_fn=data_to_evaluate.iloc[i]["id"]
            prompt = data_to_evaluate.iloc[i]["prompt"]
            correct_answer = data_to_evaluate.iloc[i]["correct_answer"]
            print(f"Prompt: {prompt}")
            print(f"Correct Answer: {correct_answer}")
            outputs = inf_learn.blurr_summarize(prompt, early_stopping=True, num_beams=4, num_return_sequences=1, max_length=1000)
            print(f"Generated Answer: {outputs[0]}")
            with open(os.path.join(base_path, f"_{epoch_current_count}_epochs_batchsize_1_results.csv"), 'a') as f:
                answer_no_non_programmatic_len_limit, answer_len_limit_by_prediction, answer_grid_size_by_prediction, output_len, other_preds, error = parse_answers_model1(outputs)
                correct_answer=correct_answer.strip().replace(".", "") #.replace(" ", "").replace(".", "")
                correct_other_preds = correct_answer.split(" ")[0:grid_starting_row]
                correct_len = int(correct_other_preds[0])
                #drop first item (length of answer)
                correct_other_preds = correct_other_preds[1:]
                #remove other preds from the correct answer
                correct_answer = correct_answer.split(" ")[grid_starting_row:len(correct_answer.split(" "))]
                correct_answer = " ".join(correct_answer)

                #don't calc any stats if there is an error
                if error==False:
                    #make a boolean comparison of all items
                    correct_other_preds_bool = [1 if x == y else 0 for x, y in zip(correct_other_preds, other_preds)]
                    output_val_len_limited_by_correct_answer=answer_no_non_programmatic_len_limit[:len(correct_answer)]
                    #convert boolean to number for csv
                    correct_answer_bylen_bool=1 if correct_answer==output_val_len_limited_by_correct_answer else 0
                    correct_answer_bool=1 if correct_answer==answer_no_non_programmatic_len_limit else 0
                    correct_answer_bypredlen_bool=1 if correct_answer==answer_len_limit_by_prediction else 0
                    correct_answer_bypredgrid_bool=1 if correct_answer==answer_grid_size_by_prediction else 0
                    sum_correct_answer+=correct_answer_bool
                    sum_correct_answer_by_actual_len+=correct_answer_bylen_bool
                    sum_correct_answer_pred_len+=correct_answer_bypredlen_bool
                    sum_correct_answer_by_pred_grid_size+=correct_answer_bypredgrid_bool
                    correct_answer_len=1 if correct_len==output_len else 0
                    sum_correct_answer_len+=correct_answer_len
                    sum_other_preds=[x+y for x, y in zip(sum_other_preds, correct_other_preds_bool)]
                    #if writing the first line, write the headers
                    if i==0:
                        if grid_starting_row>1:
                        # remove the curly braces for the headers
                            f.write('{arc_fn},{prompt},{correct_answer},{answer_no_non_programmatic_len_limit},{answer_len_limit_by_prediction},{answer_grid_size_by_prediction},{output_val_len_limited_by_correct_answer},{correct_len},{output_len},{correct_answer_len},{other_preds_string},,{correct_other_preds_string},,{correct_other_preds_bool_string},{correct_answer_bool},{correct_answer_bylen_bool},{correct_answer_bypredgrid_bool},{correct_answer_bypredlen_bool},{outputs}\n'.replace("{", "").replace("}", ""))
                        else:
                            f.write('{arc_fn},{prompt},{correct_answer},{answer_no_non_programmatic_len_limit},{answer_len_limit_by_prediction},{output_val_len_limited_by_correct_answer},{correct_len},{output_len},{correct_answer_len},{correct_answer_bool},{correct_answer_bylen_bool},{correct_answer_bypredlen_bool},{outputs}\n'.replace("{", "").replace("}", ""))
                    other_preds_string = ",".join(str(x) for x in other_preds)
                    correct_other_preds_string = ",".join(str(x) for x in correct_other_preds)
                    correct_other_preds_bool_string = ",".join(str(x) for x in correct_other_preds_bool)
                    if grid_starting_row>1:
                        #{other_preds_string},{correct_other_preds_string},{sum_other_preds_string}
                        string_to_write=f"{arc_fn},{prompt},{correct_answer},{answer_no_non_programmatic_len_limit},{answer_len_limit_by_prediction},{answer_grid_size_by_prediction},{output_val_len_limited_by_correct_answer},{correct_len},{output_len},{correct_answer_len},{other_preds_string},{correct_other_preds_string},{correct_other_preds_bool_string},{correct_answer_bool},{correct_answer_bylen_bool},{correct_answer_bypredgrid_bool},{correct_answer_bypredlen_bool},{outputs}\n"
                    else:
                        string_to_write=f"{arc_fn},{prompt},{correct_answer},{answer_no_non_programmatic_len_limit},{answer_len_limit_by_prediction},{output_val_len_limited_by_correct_answer},{correct_len},{output_len},{correct_answer_len},{correct_answer_bool},{correct_answer_bylen_bool},{correct_answer_bypredlen_bool},{outputs}\n"
                    string_to_write=string_to_write.encode('ascii', 'ignore').decode('ascii') #occasionally crashed in the past due to unicode characters
                    f.write(string_to_write)
                #write the sums
        with open(os.path.join(base_path, f"_{epoch_current_count}_epochs_batchsize_1_results.csv"), 'a') as f:
            #replace the unicode characters with ascii in string_to_write
            sum_other_preds_string = ",".join(str(x) for x in sum_other_preds)
            #create a string with grid_starting_row -1 commas
            other_preds_commas = ",".join(str(x) for x in range(grid_starting_row-1))
            string_to_write="\n{sum_other_preds_string},,{sum_correct_answer_len},{sum_correct_answer},{sum_correct_answer_by_actual_len},{sum_correct_answer_pred_len},{sum_correct_answer_by_pred_grid_size}\n".replace("{", "").replace("}", "")
            f.write(string_to_write)
            string_to_write=f"{sum_other_preds_string},{sum_correct_answer_len},{sum_correct_answer},{sum_correct_answer_by_actual_len},{sum_correct_answer_pred_len},{sum_correct_answer_by_pred_grid_size}\n"
            f.write(string_to_write)
    return sum_correct_answer, sum_correct_answer_by_actual_len, sum_correct_answer_pred_len, sum_correct_answer_by_pred_grid_size, sum_correct_answer_len, sum_other_preds_string, error

# Functions to Train and Save Overall Custom Stats

In [ ]:
def train_for_epochs_and_calc_totals(epochs_to_train, epoch_current_count):
    print(f"training for {epochs_to_train} epochs with batch size {batch_size}")    
    epoch_current_count+=epochs_to_train
    learn.fit_one_cycle(epochs_to_train, lr_max=lr, cbs=fit_cbs)
    
    current_correct_count, current_correct_by_len_count, correct_answer_by_predlen_count, current_correct_by_grid_size_count, correct_pred_len, correct_other_pred, error = get_and_save_predictions()
    if error==False:
        correct_counts.append(current_correct_count)
        correct_counts_len.append(current_correct_by_len_count)
        epoch_current.append(epoch_current_count)
        correct_counts_pred_len.append(correct_pred_len)
        correct_counts_bypred_len.append(correct_answer_by_predlen_count)
        correct_counts_other_preds.append(correct_other_pred)
        correct_counts_by_grid_size.append(current_correct_by_grid_size_count)
        hf_model.save_pretrained(f"{experiment_folder}")
        hf_tokenizer.save_pretrained(f"{experiment_folder}")

def save_data_and_stats():
    #write the correct_counts and correct_counts_len to a file final_results.csv
    with open(os.path.join(experiment_folder, f"final_results.csv"), 'a') as f:
        #write the column labels
        f.write('pretrained_model_name,epoch_start,epoch_end,epoch_step,batch_size,lr,epoch_step')
        f.write(f"{pretrained_model_name},{epoch_start},{epoch_end},{epoch_step},{batch_size},{lr},{epoch_step}\n")
        # label the columns
        f.write(f"epoch,correct_count,correct_count_by_len,correct_by_pred_len,correct_pred_len,correct_answer_by_grid_size,correct_other_pred\n")
        for i in range(len(correct_counts)):
            epochval=epoch_current_count
            try:
                curr_pred_len_correct=correct_counts_pred_len[i]
            except:
                curr_pred_len_correct=0
            f.write(f"{epochval},{correct_counts[i]},{correct_counts_len[i]},{correct_counts_bypred_len[i]},{correct_counts_pred_len[i]},{correct_counts_by_grid_size[i]},{correct_counts_other_preds[i]}\n")
        f.write("\n")

# Train the Model

You can set the model at the top.  The large-cnn version was trained as a text summarization model.  You can also load a model that you have fine tuned by specifying the folder for the huggingface model.

The training will train for epoch_steps and then will run the validation data through the model to generate the outputs and custom stats.  This is saved to a file after every epoch_steps.  The model is also saved after every epoch_steps.  The huggingface model is used for fine tuning and the pickel file is used for inference.

In [ ]:

#from transformers import BartForConditionalGeneration
pretrained_model_name = "facebook/bart-large-cnn" # 400M parameters
#pretrained_model_name = "facebook/bart-large" # 400M parameters
#pretrained_model_name = "facebook/bart-base" # 140M parameters
#uncomment the next line to load saved model and tokenizer
#pretrained_model_name=f"{experiment_folder}"
hf_arch, hf_config, hf_tokenizer, hf_model = get_hf_objects(pretrained_model_name, model_cls=BartForConditionalGeneration)
hf_arch, type(hf_config), type(hf_tokenizer), type(hf_model)
#hf_config

text_gen_kwargs = default_text_gen_kwargs(hf_config, hf_model, task='summarization')
text_gen_kwargs['min_length']=1
text_gen_kwargs['max_length']=1000
text_gen_kwargs
hf_batch_tfm = Seq2SeqBatchTokenizeTransform(
    hf_arch, hf_config, hf_tokenizer, hf_model, max_length=max_input_length, max_tgt_length=1000, text_gen_kwargs=text_gen_kwargs
)
blocks = (Seq2SeqTextBlock(batch_tokenize_tfm=hf_batch_tfm), noop)
dblock = DataBlock(blocks=blocks, get_x=ColReader('prompt'), get_y=ColReader('correct_answer'))
dls = dblock.dataloaders(df, bs=batch_size)
dls.train.items=df
dls.train.items.head(5)
dls.valid.items=df2
b = dls.one_batch()
len(b), b[0]['input_ids'].shape, b[1].shape
seq2seq_metrics = {
        'rouge': {
            'compute_kwargs': { 'rouge_types': ["rouge1", "rouge2", "rougeL"], 'use_stemmer': True },
            'returns': ["rouge1", "rouge2", "rougeL"]
        },
        'bertscore': {
            'compute_kwargs': { 'lang': 'en' },
            'returns': ["precision", "recall", "f1"]
        }
    }

model = BaseModelWrapper(hf_model)
learn_cbs = [BaseModelCallback]
fit_cbs = [Seq2SeqMetricsCallback(custom_metrics=seq2seq_metrics)]
learn = Learner(dls, 
                model,
                opt_func=ranger,
                loss_func=CrossEntropyLossFlat(),
                cbs=learn_cbs,
                splitter=partial(blurr_seq2seq_splitter, arch=hf_arch))
learn.create_opt() 
learn.freeze()

# Set max_split_size_mb to avoid memory fragmentation
dls.max_split_size_mb = 100

correct_counts=[]
correct_counts_len=[]
correct_counts_bypred_len=[]
correct_counts_pred_len=[]
correct_counts_other_preds=[]
correct_counts_by_grid_size=[]
epoch_current=[]

if epoch_start_up>0:
    epoch_current_count=epoch_start_up
    train_for_epochs_and_calc_totals(epoch_start_up, epoch_current_count)

for epochs in range(epoch_start+1, epoch_end, epoch_step):
    epoch_current_count+=epoch_step
    train_for_epochs_and_calc_totals(epoch_step, epoch_current_count)

save_data_and_stats()
#learn.validate(1, dl=None, cbs=fit_cbs)

Due to IPython and Windows limitation, python multiprocessing isn't available now.
So `number_workers` is changed to 0 to avoid getting stuck
training for 10 epochs with batch size 1


epoch,train_loss,valid_loss,rouge1,rouge2,rougeL,bertscore_precision,bertscore_recall,bertscore_f1,time
0,2.272146,2.269009,0.147694,0.051688,0.114031,0.862800,0.869021,0.865517,18:21
1,2.211774,2.120725,0.173559,0.068128,0.139230,0.877359,0.872517,0.874602,15:43
2,1.939902,2.013180,0.202258,0.089404,0.168754,0.870975,0.874913,0.872468,18:14
3,1.851954,2.024859,0.167120,0.066093,0.140591,0.868539,0.878172,0.873007,19:48
4,1.620080,1.941620,0.201064,0.094174,0.173686,0.868355,0.881470,0.874625,21:10
5,1.395362,1.965808,0.181092,0.082389,0.159703,0.864245,0.883649,0.873605,20:56
6,1.258962,1.941785,0.202341,0.103775,0.185802,0.872261,0.884262,0.877982,19:43
7,1.292982,1.979436,0.208461,0.110302,0.190801,0.873430,0.885956,0.879447,20:48


# Run all above cells for fine tuning.  The cells below can be used to resume training if it is stopped

# Tools Below

In [ ]:
# use this to continue trianing if the above cell fails or is stopped before completion
epoch_start=38
lr = 3.1e-6 # learning rate
epoch_end=109 # ending epoch number for training
for epochs in range(epoch_start+1, epoch_end, epoch_step):
    epoch_current_count+=epoch_step
    train_for_epochs_and_calc_totals(epoch_step, epoch_current_count)

save_data_and_stats()

If you stop training early, you can use the below to run the model against the validation dataset and it will save the results to a file.

In [ ]:
get_and_save_predictions()

You can use the code below to export a pickel file for inference.  Note that this won't work across platform.  If you train on Windows, it won't work on Linux and vice versa.

In [ ]:
#learn.metrics = None
#picklefn=f"{experiment_folder}ft_cnndm_export_{batch_size_start}_bs_start_{batch_size_end}_{epoch_current_count}_epochs_per_bs_{lr}_lr_{len(df)}_items.pkl"
#learn.export(fname=picklefn)


If you stop in the middle of a training set, you can save the Huggingface model and tokenizer with the code below.  This model does work across platforms.

In [ ]:

#use this to save the huggingface model and tokenizer
hf_model.save_pretrained(f"{experiment_folder}/model")
hf_tokenizer.save_pretrained(f"{experiment_folder}/model")

('.//model20\\tokenizer_config.json',
 './/model20\\special_tokens_map.json',
 './/model20\\vocab.json',
 './/model20\\merges.txt',
 './/model20\\added_tokens.json',
 './/model20\\tokenizer.json')